<a href="https://colab.research.google.com/github/sdikici/Demand_Forecasting_Prophet_DeepAR/blob/main/DeepAR_hyperparameter_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d sercandikici/merged-dataset-electricty-weather-for-modelling
! unzip merged-dataset-electricty-weather-for-modelling.zip

Dataset URL: https://www.kaggle.com/datasets/sercandikici/merged-dataset-electricty-weather-for-modelling
License(s): unknown
100% 104k/104k [00:00<00:00, 382kB/s]
100% 104k/104k [00:00<00:00, 381kB/s]
Archive:  merged-dataset-electricty-weather-for-modelling.zip
  inflating: merged_data.csv         


In [3]:
!pip install gluonts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.2 MB/s eta 0:00:00


In [4]:
pip install mxnet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [5]:
pip install mxnet --upgrade

In [6]:
import numpy as np
np.bool = np.bool_

In [13]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon
import json
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.mx import DeepAREstimator
from gluonts.mx.trainer import Trainer
import itertools

In [8]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true))
    return mape

def root_mean_squared_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

def r_squared(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mean_y_true = np.mean(y_true)
    ss_total = np.sum((y_true - mean_y_true) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_residual / ss_total)
    return r2

In [ ]:
mx.random.seed(7)
np.random.seed(7)

In [9]:
df_merged = pd.read_csv("merged_data.csv")
df_merged['settlement_date'] = pd.to_datetime(df_merged['settlement_date'])

,settlement_date,tsd,is_holiday,temp
0,2021-01-01 00:00:00,28552.00,1.0,1.55
1,2021-01-01 02:00:00,26579.75,1.0,1.60
2,2021-01-01 04:00:00,25236.00,1.0,0.55
3,2021-01-01 06:00:00,25792.25,1.0,-0.35
4,2021-01-01 08:00:00,28162.75,1.0,-0.10
...,...,...,...,...
13136,2023-12-31 16:00:00,34006.00,0.0,7.60
13137,2023-12-31 18:00:00,32273.00,0.0,8.20
13138,2023-12-31 20:00:00,27456.50,0.0,8.45
13139,2023-12-31 22:00:00,24139.25,0.0,8.35


In [10]:
df_model_1 = df_merged[["tsd","settlement_date"]]
df_model_1= df_model_1.set_index("settlement_date")
split_from = 90*12 #train test split is from 90days

train_data = df_model_1[:-split_from]
test_data = df_model_1[-split_from:]

In [11]:
freq = "2H"
days_to_predict=7
prediction_length = days_to_predict*12 #predicting 7 days
context_length = prediction_length *3

In [16]:
# Define the parameter grid for DeepAR
param_grid = {
    'epochs': [10, 15],
    'learning_rate': [0.0001, 0.001, 0.1],
    'num_cells': [32, 64],
    'num_layers': [2, 4]
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mapes = []  # Store the MAPEs for each parameter combination here

# Use cross validation to evaluate all parameters
for params in all_params:
    # Define DeepAR estimator with given parameters
    estimator = DeepAREstimator(
        freq=freq,
        context_length=context_length,
        prediction_length=prediction_length,
        num_layers=params['num_layers'],
        num_cells=params['num_cells'],
        trainer=Trainer(
            epochs=params['epochs'],
            learning_rate=params['learning_rate']
        )
    )

    subset = test_data[test_data.index <= "2023-10-10 02:00:00"]

    train_ds = ListDataset(
    [{"start":train_data.index[0],
      "target":train_data.tsd}],
    freq=freq
)

    test_ds = ListDataset(
        [{"start":subset.index[0],
          "target":subset.tsd}],
        freq=freq
    )


    # Train the model
    m = estimator.train(training_data=train_ds)

    # Generate forecasts
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,
        predictor=m,
        num_samples=100
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    # Get MAPE for the forecasts
    evaluator = Evaluator()
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts))
    mape = agg_metrics["MAPE"]
    mapes.append(mape)

# Combine parameters and corresponding MAPEs into a DataFrame
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
print(tuning_results)


100%|██████████| 50/50 [00:22<00:00,  2.27it/s, epoch=10/10, avg_epoch_loss=8.97]
Running evaluation: 1it [00:00,  4.00it/s]
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
100%|██████████| 50/50 [00:40<00:00,  1.22it/s, epoch=10/10, avg_epoch_loss=8.85]
Running evaluation: 1it [00:00,  3.88it/s]
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:1

    epochs  learning_rate  num_cells  num_layers      mape
0       10         0.0001         32           2  0.266020
1       10         0.0001         32           4  0.272839
2       10         0.0001         64           2  0.351391
3       10         0.0001         64           4  0.273766
4       10         0.0010         32           2  0.194069
5       10         0.0010         32           4  0.166846
6       10         0.0010         64           2  0.218551
7       10         0.0010         64           4  0.185844
8       10         0.1000         32           2  0.204599
9       10         0.1000         32           4  0.262067
10      10         0.1000         64           2  9.352641
11      10         0.1000         64           4  1.647865
12      15         0.0001         32           2  0.474402
13      15         0.0001         32           4  0.259949
14      15         0.0001         64           2  0.208121
15      15         0.0001         64           4  0.1870


/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


In [17]:
tuning_results

,epochs,learning_rate,num_cells,num_layers,mape
0,10,0.0001,32,2,0.266020
1,10,0.0001,32,4,0.272839
2,10,0.0001,64,2,0.351391
3,10,0.0001,64,4,0.273766
4,10,0.0010,32,2,0.194069
5,10,0.0010,32,4,0.166846
6,10,0.0010,64,2,0.218551
7,10,0.0010,64,4,0.185844
8,10,0.1000,32,2,0.204599
9,10,0.1000,32,4,0.262067
